<a href="https://colab.research.google.com/github/ngoan22mse23088/python4engineers-assignments/blob/master/Assignments2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mysql-connector-python
!apt-get install mysql-server > /dev/null
!service mysql start

!mysql -u root -e "SELECT 'Connected';"
!mysql -u root -e "CREATE DATABASE mydatabase;"
!mysql -u root -e "CREATE USER 'root'@'localhost' IDENTIFIED BY 'root';"
!mysql -u root -e "GRANT ALL PRIVILEGES ON mydatabase.* TO 'root'@'localhost';"
!mysql -u root -e "SELECT User FROM mysql.user;"
!mysql -u root -e "SHOW DATABASES;"
!mysql -u root -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root';"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
W: Operation was interrupted before it could finish
^C
 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.
Enter password: 

In [ ]:
# Thêm các thư viện cơ bản để xử lý
from flask import Flask, flash, render_template, request
import mysql.connector

app = Flask(__name__)
app.secret_key = "assignment3"

class Database:
    # Hàm kết nối
    def connect(self):
        return pymysql.connect(host="localhost", user="root", password="12345678", database="mydatabase", charset='utf8mb4')

    # Hàm đọc data table
    def read(self, id):
        con = Database.connect(self)
        cursor = con.cursor()

        try:
            if id == None:
                cursor.execute("SELECT * FROM posts order by id asc")
            else:
                cursor.execute(
                    "SELECT * FROM posts where id = %s order by id asc", (id))

            return cursor.fetchall()
        except:
            return ()
        finally:
            con.close()

    # Hàm get data từ table posts
    def get_posts(self):
        con = Database.connect(self)
        cursor = con.cursor()

        try:
            cursor.execute("SELECT id, title, content FROM posts")
            posts = cursor.fetchall()
            return [Post(*post) for post in posts]
        except:
            return ()
        finally:
            con.close()
        pass

    # Hàm get post theo id
    def get_post_by_id(self, id):
        con = Database.connect(self)
        cursor = con.cursor()

        try:
            cursor.execute(
                "SELECT id, title, content FROM posts WHERE id = %s", (id))
            post = cursor.fetchone()
            return Post(post[0], post[1], post[2])
        except:
            return None
        finally:
            con.close()
        pass

    # Hàm thêm mới 1 post
    def insert(self, post):
        con = Database.connect(self)
        cursor = con.cursor()

        try:
            cursor.execute("INSERT INTO posts(title, content) VALUES(%s, %s)",
                           (post.title, post.content))
            con.commit()

            return True
        except:
            con.rollback()

            return False
        finally:
            con.close()

    # Hàm cập nhật 1 post
    def update(self, post):
        con = Database.connect(self)
        cursor = con.cursor()

        try:
            cursor.execute("UPDATE posts set title = %s, content = %s where id = %s",
                           (post.title, post.content, post.id))
            con.commit()

            return True
        except:
            con.rollback()

            return False
        finally:
            con.close()

    # Hàm xóa 1 post theo id
    def delete(self, id):
        con = Database.connect(self)
        cursor = con.cursor()

        try:
            cursor.execute("DELETE FROM posts where id = %s", (id,))
            con.commit()

            return True
        except:
            con.rollback()

            return False
        finally:
            con.close()

class Post:
    def __init__(self, id, title, content):
        self.id = id
        self.title = title
        self.content = content

db = Database()

# Kết nối tới MySQL (port mặc định 3306)
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="12345678",
    database="mydatabase"
)

# Tạo table nếu chưa có
mycursor = mydb.cursor()
mycursor.execute(
    "CREATE TABLE IF NOT EXISTS posts (id int NOT NULL AUTO_INCREMENT,title varchar(255),content varchar(255),PRIMARY KEY (id));")
mycursor.execute("SELECT * FROM posts")
myresult = mycursor.fetchall()

# Trang main


@app.route('/')
def index():
    return render_template('index.html')

# Trang create


@app.route('/create', methods=['GET'])
def create():
    return render_template('create.html')

# Trang create


@app.route('/create', methods=['POST'])
def addPost():
    form = request.form
    title = form.get('title')
    content = form.get('content')
    post = Post(None, title, content)
    if db.insert(post):
        flash("Post has been added")
    else:
        flash("Post can not be added")
    return posts()

# Trang lấy 1 post theo id


@app.route('/edit/<int:id>/', methods=['GET'])
def edit(id):
    post = db.get_post_by_id(id)
    return render_template('edit.html', post=post)

# Trang cập nhât 1 post theo id


@app.route('/edit/<int:id>/', methods=['POST'])
def update_edit(id):
    form = request.form
    title = form.get('title')
    content = form.get('content')
    post = Post(id, title, content)
    if db.update(post):
        flash("A post has been updated")
    else:
        flash("A post can not be updated")
    post = db.get_post_by_id(id)
    return render_template('edit.html', post=post)

# Trang get ds post theo id


@app.route('/posts', methods=['GET'])
def posts():
    posts = db.get_posts()
    return render_template('posts.html', posts=posts)

# Trang xóa 1 post theo id


@app.route('/delete/<int:id>', methods=['POST'])
def delete(id):
    if db.delete(id):
        flash("A post has been deleted")
    else:
        flash("A post can not be deleted")
    return posts()

# Trang xử lý khi not found


@app.errorhandler(404)
def page_not_found(error):
    return render_template('error.html')


if __name__ == '__main__':
    app.run(port=8080, host="0.0.0.0")


: 

In [2]:
!pip install flask
!pip install colabcode
!pip install flask_mysqldb
!pip install mysql-connector-python
!pip install flask-ngrok
!pip install pyngrok
!ngrok http 5000

Traceback (most recent call last):
  File "/usr/lib/python3.9/subprocess.py", line 1189, in wait
    return self._wait(timeout=timeout)
  File "/usr/lib/python3.9/subprocess.py", line 1917, in _wait
    (pid, sts) = self._try_wait(0)
  File "/usr/lib/python3.9/subprocess.py", line 1875, in _try_wait
    (pid, sts) = os.waitpid(self.pid, wait_flags)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/ngrok", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.9/dist-packages/pyngrok/ngrok.py", line 517, in main
    run(sys.argv[1:])
  File "/usr/local/lib/python3.9/dist-packages/pyngrok/ngrok.py", line 505, in run
    process.run_process(pyngrok_config.ngrok_path, args)
  File "/usr/local/lib/python3.9/dist-packages/pyngrok/process.py", line 347, in run_process
    subprocess.call(start)
  File "/usr/lib/python3.9/subprocess.py", line 351, in call
    return p.wait(timeout=t

In [ ]:
!ngrok config add-authtoken 2NwsOLXovIUtXLuIqCCHXtJedAo_4cA58oGig9qeTAeqWiguh

In [ ]:
from flask import Flask
from pyngrok import ngrok

app = Flask(__name__)

public_url = ngrok.connect(5000).public_url
print(f" * Running on {public_url}")

@app.route("/")
def home():    
    return "Hello, World!"

if __name__ == "__main__":
    app.run()

In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(5000).public_url
print(f" * Running on {public_url}")

In [6]:
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok
import mysql.connector

app = Flask(__name__)

run_with_ngrok(app) 

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="mydatabase"
)

@app.route('/posts', methods=['GET'])
def get_posts():
    cursor = db.cursor()
    cursor.execute("SELECT * FROM posts")
    result = cursor.fetchall()
    posts = []
    for post in result:
        posts.append({
            'id': post[0],
            'created': post[1],
            'title': post[2],
            'content': post[3]
        })
    return jsonify(posts)

@app.route('/posts', methods=['POST'])
def create_post():
    cursor = db.cursor()
    title = request.json['title']
    content = request.json['content']
    cursor.execute("INSERT INTO posts (title, content) VALUES (%s, %s)", (title, content))
    db.commit()
    return jsonify({'message': 'Post created successfully'})

@app.route('/posts/<int:id>', methods=['GET'])
def get_post(id):
    cursor = db.cursor()
    cursor.execute("SELECT * FROM posts WHERE id = %s", (id,))
    result = cursor.fetchone()
    if result is None:
        return jsonify({'message': 'Post not found'}), 404
    post = {
        'id': result[0],
        'created': result[1],
        'title': result[2],
        'content': result[3]
    }
    return jsonify(post)

@app.route('/posts/<int:id>', methods=['PUT'])
def update_post(id):
    cursor = db.cursor()
    title = request.json['title']
    content = request.json['content']
    cursor.execute("UPDATE posts SET title = %s, content = %s WHERE id = %s", (title, content, id))
    db.commit()
    return jsonify({'message': 'Post updated successfully'})

@app.route('/posts/<int:id>', methods=['DELETE'])
def delete_post(id):
    cursor = db.cursor()
    cursor.execute("DELETE FROM posts WHERE id = %s", (id,))
    db.commit()
    return jsonify({'message': 'Post deleted successfully'})

if __name__ == '__main__':
    # app.run()
    app.run(debug=True, port=8181)


DatabaseError: ignored